# Aggregating DataFrames in PySpark HW

First let's start up our PySpark instance

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark= SparkSession.builder.appName('aggregator').getOrCreate()
spark

## Read in the dataFrame for this Notebook

In [2]:
airbnb = spark.read.csv('Datasets/nyc_air_bnb.csv',inferSchema=True,header=True)

## About this dataset

This dataset describes the listing activity and metrics for Air BNB bookers in NYC, NY for 2019. Each line in the dataset is a booking. 

**Source:** https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data/data

Let's go ahead and view the first few records of the dataset so we know what we are working with.

In [3]:
airbnb.limit(5).toPandas()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,None,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Now print the schema so we can make sure all the variables have the correct types

In [4]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: string (nullable = true)
 |-- minimum_nights: string (nullable = true)
 |-- number_of_reviews: string (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: string (nullable = true)
 |-- calculated_host_listings_count: string (nullable = true)
 |-- availability_365: integer (nullable = true)



Notice here that some of the columns that are obviously numeric have been incorrectly identified as "strings". Let's edit that. Otherwise we cannot aggregate any of the numeric columns.

In [6]:
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [7]:
df = airbnb.withColumn('price', airbnb['price'].cast(IntegerType()))\
        .withColumn('minimum_nights', airbnb['minimum_nights'].cast(IntegerType()))\
        .withColumn('number_of_reviews', airbnb['number_of_reviews'].cast(IntegerType()))\
        .withColumn('reviews_per_month', airbnb['reviews_per_month'].cast(IntegerType()))\
        .withColumn('calculated_host_listings_count', airbnb['calculated_host_listings_count'].cast(IntegerType()))

print(df.printSchema())

df.limit(5).toPandas()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)

None


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.0,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.0,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.9419,Private room,150,3,0,None,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.0,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.0,1,0


### Alright now we are ready to dig in!


### 1. How many rows are in this dataset?

In [8]:
df.count()

49079

### 2. How many total reviews does each host have?

In [19]:
df.groupBy('host_name').sum('number_of_reviews').sort(col('sum(number_of_reviews)').desc()).limit(100).toPandas()

,host_name,sum(number_of_reviews)
0,Michael,11081
1,David,8100
2,John,7223
3,Jason,6522
4,Alex,6204
...,...,...
95,Anne,1606
96,Sonia,1549
97,Ian,1541
98,Carlos,1537


### 3. Show the min and max of all the numeric variables in the dataset

In [23]:
df.select('price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month','calculated_host_listings_count', 'availability_365').summary( "min", "max").limit(20).toPandas()

,summary,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,min,-74,0,0,0,0,0
1,max,10000,1250,629,58,365,365


### 4. Which host had the highest number of reviews?

Only display the top result.

Bonus: format the column names

In [24]:
df.groupBy('host_name').sum('number_of_reviews').sort(col('sum(number_of_reviews)').desc()).limit(1).toPandas()

,host_name,sum(number_of_reviews)
0,Michael,11081


### 5. On average, how many nights did most hosts specify for a minimum?

In [25]:
df.select(avg('minimum_nights')).limit(5).toPandas()

,avg(minimum_nights)
0,7.128613


### 6. What is the most expensive neighborhood to stay in on average?

Note: only show the one result

In [33]:
df.groupBy('neighbourhood').agg(avg(df['price']).alias('avg_price_per_night')).sort(col('avg_price_per_night').desc()).limit(11).toPandas()

,neighbourhood,avg_price_per_night
0,Fort Wadsworth,800.000000
1,Woodrow,700.000000
2,Sea Gate,548.333333
3,Tribeca,490.638418
4,Riverdale,442.090909
5,Prince's Bay,409.500000
6,Battery Park City,367.557143
7,Randall Manor,352.944444
8,Flatiron District,341.925000
9,NoHo,297.855263


### 7. Display a two by two table that shows the average prices by room type (private and shared only) and neighborhood group (Manhattan and Brooklyn only)

In [34]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: integer (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)



In [52]:
[item for item in df.toPandas()['room_type'].unique() if ]

array(['Private room', 'Entire home/apt', 'Shared room', None, '89', '99',
       '225', '56', '69', '170', '-73.95777', '298', '59', '50', '80',
       '65', '-73.95179', '160', '-73.94639', '140', '200', '45', '67',
       '86', '72', '95', '230', '110', 'Howard', '250', '119', '138',
       '350', '150', '93', '188', '60', '40', '333', '63', '100', '88',
       '300', '425', '175', '159', '-73.90783', '55', '79', '31', '75',
       '120', '116', '410', '135', '40.71509', '70', '46', '128', '38',
       '81', '35', '54', '109', '145', '179', '185', '6500', '42', '105',
       '130', '180', '205', '85', '39', '-73.94861', '279', '57', '51',
       '40.57453', '299', '78', '48', '-74.00244', '249', '68', '115'],
      dtype=object)

In [55]:
df.filter((df['room_type']== 'Private room')|
          (df['room_type']== 'Entire home/apt')|
          (df['room_type']== 'Shared room'))\
.groupBy('room_type')\
.pivot("neighbourhood_group", ["Manhattan", "Brooklyn"])\
.agg(avg('price')).limit(10).toPandas()

,room_type,Manhattan,Brooklyn
0,Shared room,89.069038,50.527845
1,Entire home/apt,249.314075,178.377792
2,Private room,116.054003,76.472340


In [56]:
df.filter((df['room_type']== 'Private room')|
          (df['room_type']== 'Shared room'))\
.groupBy('room_type')\
.pivot("neighbourhood_group", ["Manhattan", "Brooklyn"])\
.agg(avg('price')).limit(10).toPandas()

,room_type,Manhattan,Brooklyn
0,Shared room,89.069038,50.527845
1,Private room,116.054003,76.472340


### Alright that's all folks!

### Great job!